In [12]:
import pandas as pd
import numpy as np
import pickle
pd.__version__

'2.2.2'

In [14]:
with open('data/movement_df_with_ld_validation.pkl', 'rb') as f:
    movement_df = pickle.load(f)
movement_df # without dropping suspicious movements

,author_id,move_year,institution_id,move_time,prev_ins,stay_time,city,country_code,continent_code,prev_country,level_distance,work,coauthors,field,domain,author_validated
0,A5000000072,2019,I198243066,2019-01-01,I153718931,NaN,Heredia,CR,NaN,CA,2,0,0,2,1,3
1,A5000000092,2002,I74760111,2002-11-23,I131249849,4057 days 00:00:00,Dallas,US,NaN,US,1,0,0,1,1,2
2,A5000000092,2014,I16076960,2014-01-01,I74760111,2475 days 00:00:00,Islamabad,PK,AS,US,3,0,0,1,1,2
3,A5000000092,2020,I131249849,2020-10-11,I16076960,NaN,Corvallis,US,NaN,PK,3,1,5,1,1,8
4,A5000000103,1992,I4210135003,1992-05-01,I88155538,11644 days 00:00:00,Pamplona,ES,EU,ES,0,0,1,2,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5194551,A5097964981,2004,I4210139845,2004-03-01,I904495901,NaN,Bochum,DE,EU,DE,0,0,0,2,3,5
5194552,A5097991687,1993,I2801112126,1993-08-01,I4210122458,NaN,Bern,CH,EU,CH,0,0,0,2,2,4
5194553,A5098096471,1985,I4210159963,1985-11-01,I2801942218,NaN,Copenhagen,DK,EU,DK,1,1,0,1,2,4
5194554,A5098805673,2015,I80043,2015-12-01,I4210147769,NaN,Paris,FR,EU,ES,2,23,0,3,3,29


In [15]:
movement_df['verifiation_score'] = movement_df['work'] + movement_df['coauthors'] + movement_df['field']
verified_df = movement_df[movement_df['verifiation_score'] > 0 ]
len(verified_df) / len(movement_df)

0.9927653104519424

In [17]:

# convert the NaNs in the "continent_code" to string "NA"
mdf = verified_df

mdf['continent_code'] = mdf['continent_code'].fillna('NA')


mdf['move_time'] = pd.to_datetime(mdf['move_time'])

bins = list(range(1960, 2026, 5))
labels = [f'{i}-{i+4}' for i in bins[:-1]]

print(bins)
print(labels)

[1960, 1965, 1970, 1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2025]
['1960-1964', '1965-1969', '1970-1974', '1975-1979', '1980-1984', '1985-1989', '1990-1994', '1995-1999', '2000-2004', '2005-2009', '2010-2014', '2015-2019', '2020-2024']


/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_70169/571244635.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdf['continent_code'] = mdf['continent_code'].fillna('NA')
/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_70169/571244635.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdf['move_time'] = pd.to_datetime(mdf['move_time'])


In [4]:
labels

['1960-1964',
 '1965-1969',
 '1970-1974',
 '1975-1979',
 '1980-1984',
 '1985-1989',
 '1990-1994',
 '1995-1999',
 '2000-2004',
 '2005-2009',
 '2010-2014',
 '2015-2019',
 '2020-2024']

In [18]:
mdf[mdf['institution_id'] == mdf['prev_ins']]

,author_id,move_year,institution_id,move_time,prev_ins,stay_time,city,country_code,continent_code,prev_country,level_distance,work,coauthors,field,domain,author_validated,verifiation_score


In [19]:
mdf['time_slot']  = pd.cut(mdf['move_time'].dt.year, bins=bins, labels=labels, right=False)

movement_counts = mdf.groupby(['prev_ins', 'institution_id', 'time_slot', 'level_distance'], observed = True).size().reset_index(name='counts')

movement_counts

/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_70169/1711954190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdf['time_slot']  = pd.cut(mdf['move_time'].dt.year, bins=bins, labels=labels, right=False)


,prev_ins,institution_id,time_slot,level_distance,counts
0,I100005738,I100191712,2005-2009,1,1
1,I100005738,I10052268,2015-2019,1,1
2,I100005738,I10052268,2020-2024,1,1
3,I100005738,I100538780,1995-1999,1,1
4,I100005738,I102149020,2015-2019,1,1
...,...,...,...,...,...
3193144,I99981631,I4210099175,1990-1994,0,1
3193145,I99981631,I4210099175,2000-2004,0,3
3193146,I99981631,I4210146600,2015-2019,0,1
3193147,I99981631,I4210146600,2020-2024,0,1


In [20]:
import tqdm

# 预计算总的移动数量
total_movements = mdf.groupby('time_slot').size().to_dict()

# 预计算每个时间段和前机构的流出数量
outflows = mdf.groupby(['time_slot', 'prev_ins']).size().to_dict()

# 预计算每个时间段和当前机构的流入数量
inflows = mdf.groupby(['time_slot', 'institution_id']).size().to_dict()

def calculate_simulated_movement(row):
    time_slot = row['time_slot']
    prev_ins = row['prev_ins']
    institution_id = row['institution_id']

    total_movements_count = total_movements.get(time_slot, 0)
    outflow_count = outflows.get((time_slot, prev_ins), 0)
    inflow_count = inflows.get((time_slot, institution_id), 0)

    return (outflow_count * inflow_count) / total_movements_count if total_movements_count != 0 else 0

tqdm.tqdm.pandas()

movement_counts['Simulated_movement'] = movement_counts.progress_apply(calculate_simulated_movement, axis=1)

/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_70169/4117328574.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_movements = mdf.groupby('time_slot').size().to_dict()
/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_70169/4117328574.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  outflows = mdf.groupby(['time_slot', 'prev_ins']).size().to_dict()
/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_70169/4117328574.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cu

In [21]:
movement_counts['time_slot'].value_counts()

time_slot
2020-2024    734367
2015-2019    622815
2010-2014    508090
2005-2009    388317
2000-2004    283748
1995-1999    210072
1990-1994    154675
1985-1989    106114
1980-1984     75248
1975-1979     52464
1970-1974     35445
1965-1969     18644
1960-1964      3150
Name: count, dtype: int64

In [22]:
movement_counts.to_csv('data/movement_counts.csv', index=False)

In [11]:
mdf[mdf['author_id'] == 'A5040257374']

,author_id,move_year,institution_id,move_time,prev_ins,stay_time,city,country_code,continent_code,prev_country,level_distance,work,coauthors,field,domain,author_validated,time_slot
2273536,A5040257374,2023,I4210112678,2023-01-01,I76903346,NaN,Barranquilla,CO,SA,PT,3,595,100,15,4,714,2020-2024
